For the Kernel Affine Projection algorithm. We initialize the discrete-time non-linear system through 

v_n = 1.1exp(-|v_n-1|) + u_n

with the desired output d_n being

d_n = v_n^2

For initialization, we initialize u_n from a zero-mean Guassian distribution  with standard deviation 0.25.

In [3]:
import numpy as np

sigma = 0.25

np.random.normal(0,sigma)

-0.09986927357735859

For our kernel function, we will be using a modified Guassian kernel described as

k(ui,uj) = exp(-||ui - uj||/ b0)

for a varied optimized b0 from 0.1 to 1 in increments of 0.005

In [4]:
def KAPgaussianKernel(a,b):
    b0 = 0.7
    return np.linalg.norm(a-b) / b0

In [9]:
end = 500

un = np.random.normal(0,sigma,end)
v_0 = 0.5
v_1 = 1.1 * np.exp(v_0) + un


noise = np.random.normal(0,1,end)

d_0 = v_0 ** 2 + noise[0]
d_1 = v_1 ** 2 + noise[1]

v = np.array([v_0,v_1])
d = np.array([d_0,d_1])

